In [26]:
from IPython.display import Image
import pickle

# Download and unzip pickled version from here: http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = pickle.load(open('mnist.pkl', 'rb'))
print "Shapes:"
print train_x.shape, train_y.shape
print valid_x.shape, valid_y.shape
print test_x.shape, test_y.shape

# print "--------------"
# print "Example input:"
# print train_x[0]
# print "Example label:"
# print train_y[0]


Shapes:
(50000, 784) (50000,)
(10000, 784) (10000,)
(10000, 784) (10000,)


In [27]:
# Show example images - using tile_raster_images helper function from OpenDeep to get 28x28 image from 784 array.
from utils import tile_raster_images
from PIL import Image as pil_img

input_images = train_x[:25]
im = pil_img.fromarray(
    tile_raster_images(input_images, 
                       img_shape=(28, 28), 
                       tile_shape=(1, 25),
                       tile_spacing=(1, 1))
)
im.save("some_mnist_numbers.png")
Image(filename="some_mnist_numbers.png")


In [28]:
# Your basic Theano imports.
import theano
import theano.tensor as T

theano.config.floatX = 'float32'

x = T.matrix('x')


In [29]:
input_size = 28*28
hidden_nodes = 50
output_size = 10

In [30]:
# Compute the hidden layer from the input
import numpy
import numpy.random as rng

i = numpy.sqrt(6. / (input_size+hidden_nodes))
# W_x = numpy.asarray(rng.normal(loc=0.0, scale=.05, size=(input_size, hidden_nodes)), dtype=theano.config.floatX)
W_x = numpy.asarray(rng.uniform(low=-i, high=i, size=(input_size, hidden_nodes)), dtype=theano.config.floatX)
b_h = numpy.zeros(shape=(hidden_nodes,), dtype=theano.config.floatX)

W_x = theano.shared(W_x, name="W_x")
b_h = theano.shared(b_h, name="b_h")

h = T.tanh(
    T.dot(x, W_x) + b_h
)


In [31]:
# Compute the output class probabilities from the hidden layer
i = numpy.sqrt(6. / (hidden_nodes+output_size))
# W_h = numpy.asarray(rng.normal(loc=0.0, scale=.05, size=(hidden_nodes, output_size)), dtype=theano.config.floatX)
W_h = numpy.asarray(rng.uniform(low=-i, high=i, size=(hidden_nodes, output_size)), dtype=theano.config.floatX)
b_y = numpy.zeros(shape=(output_size,), dtype="float32")

W_h = theano.shared(W_h, name="W_h")
b_y = theano.shared(b_y, name="b_y")

y = T.nnet.softmax(
    T.dot(h, W_h) + b_y
)

# The actual predicted label
y_hat = T.argmax(y, axis=1)


In [32]:
# Find cost compared to correct labels
correct_labels = T.ivector("labels")

log_likelihood = T.log(y)[T.arange(correct_labels.shape[0]), correct_labels]
cost = -T.mean(log_likelihood)


In [33]:
# Compute gradient updates for the parameters
parameters = [W_x, b_h, W_h, b_y]
gradients = T.grad(cost, parameters)

learning_rate = 0.01
train_updates = [(param, param - learning_rate*gradient) for param, gradient in zip(parameters, gradients)]


In [34]:
# Compile function for training (changes parameters via updates) and testing (no updates)
f_train = theano.function(
    inputs=[x, correct_labels], 
    outputs=cost, 
    updates=train_updates, 
    allow_input_downcast=True
)

In [35]:
f_test = theano.function(
    inputs=[x], 
    outputs=y_hat, 
    allow_input_downcast=True
)

In [36]:
def run(batch_size=100, epochs=100, check_frequency=3):
    # Main training loop
    train_batches = len(train_x) / batch_size
    valid_batches = len(valid_x) / batch_size
    test_batches = len(test_x) / batch_size

    for epoch in range(epochs):
        print epoch+1, ":",

        train_costs = []
        train_accuracy = []
        for i in range(train_batches):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_labels = train_y[i*batch_size:(i+1)*batch_size]

            costs = f_train(batch_x, batch_labels)
            preds = f_test(batch_x)
            acc = sum(preds==batch_labels)/float(len(batch_labels))

            train_costs.append(costs)
            train_accuracy.append(acc)
        print "cost:", numpy.mean(train_costs), "\ttrain:", str(numpy.mean(train_accuracy)*100)+"%",

        valid_accuracy = []
        for i in range(valid_batches):
            batch_x = valid_x[i*batch_size:(i+1)*batch_size]
            batch_labels = valid_y[i*batch_size:(i+1)*batch_size]

            preds = f_test(batch_x)
            acc = sum(preds==batch_labels)/float(len(batch_labels))

            valid_accuracy.append(acc)
        print "\tvalid:", str(numpy.mean(valid_accuracy)*100)+"%",

        test_accuracy = []
        for i in range(test_batches):
            batch_x = test_x[i*batch_size:(i+1)*batch_size]
            batch_labels = test_y[i*batch_size:(i+1)*batch_size]

            preds = f_test(batch_x)
            acc = sum(preds==batch_labels)/float(len(batch_labels))

            test_accuracy.append(acc)
        print "\ttest:", str(numpy.mean(test_accuracy)*100)+"%"

        if (epoch+1) % check_frequency == 0:
            print 'saving filters...'
            weight_filters = pil_img.fromarray(
                    tile_raster_images(
                        W_x.get_value(borrow=True).T,
                        img_shape=(28, 28),
#                         tile_shape=(20, 25),
                        tile_shape=(7, 8),
                        tile_spacing=(1, 1)
                    )
                )
            weight_filters.save("mlp_filters_%d.png"%(epoch+1))

In [37]:
run(batch_size=100, epochs=200, check_frequency=3)

1 : cost: 1.11102 	train: 73.69% 	valid: 85.5% 	test: 84.6%
2 : cost: 0.601599 	train: 85.644% 	valid: 88.36% 	test: 87.83%
3 : cost: 0.485071 	train: 87.768% 	valid: 89.62% 	test: 89.13%
saving filters...
4 : cost: 0.42829 	train: 88.766% 	valid: 90.24% 	test: 89.8%
5 : cost: 0.3936 	train: 89.436% 	valid: 90.71% 	test: 90.39%
6 : cost: 0.369611 	train: 89.972% 	valid: 91.04% 	test: 90.91%
saving filters...
7 : cost: 0.351627 	train: 90.36% 	valid: 91.44% 	test: 91.15%
8 : cost: 0.337356 	train: 90.716% 	valid: 91.71% 	test: 91.39%
9 : cost: 0.325543 	train: 90.986% 	valid: 91.91% 	test: 91.57%
saving filters...
10 : cost: 0.315444 	train: 91.248% 	valid: 92.1% 	test: 91.71%
11 : cost: 0.306593 	train: 91.512% 	valid: 92.27% 	test: 91.9%
12 : cost: 0.298681 	train: 91.72% 	valid: 92.4% 	test: 92.07%
saving filters...
13 : cost: 0.291497 	train: 91.89% 	valid: 92.6% 	test: 92.3%
14 : cost: 0.284895 	train: 92.066% 	valid: 92.67% 	test: 92.41%
15 : cost: 0.278767 	train: 92.26% 	valid: 

118 : cost: 0.103302 	train: 97.276% 	valid: 96.69% 	test: 96.44%
119 : cost: 0.102706 	train: 97.292% 	valid: 96.71% 	test: 96.45%
120 : cost: 0.102117 	train: 97.316% 	valid: 96.72% 	test: 96.45%
saving filters...
121 : cost: 0.101534 	train: 97.328% 	valid: 96.72% 	test: 96.45%
122 : cost: 0.100957 	train: 97.346% 	valid: 96.73% 	test: 96.46%
123 : cost: 0.100387 	train: 97.364% 	valid: 96.73% 	test: 96.48%
saving filters...
124 : cost: 0.0998237 	train: 97.372% 	valid: 96.74% 	test: 96.48%
125 : cost: 0.0992661 	train: 97.386% 	valid: 96.77% 	test: 96.48%
126 : cost: 0.0987145 	train: 97.402% 	valid: 96.77% 	test: 96.49%
saving filters...
127 : cost: 0.0981688 	train: 97.422% 	valid: 96.76% 	test: 96.51%
128 : cost: 0.097629 	train: 97.442% 	valid: 96.82% 	test: 96.52%
129 : cost: 0.0970949 	train: 97.458% 	valid: 96.82% 	test: 96.53%
saving filters...
130 : cost: 0.0965664 	train: 97.474% 	valid: 96.83% 	test: 96.55%
131 : cost: 0.0960435 	train: 97.496% 	valid: 96.84% 	test: 96.5